# Scraping briefly.co

In [60]:
import requests
import json
import random

In [ ]:
session = requests.Session()
response = session.get('https://api.briefly.co/v1/anchors/popular?limit=30')
anchors = response.json()
users = set()
for anch in anchors:
    response = requests.get('https://api.briefly.co/v1/anchor/' + anch['id'] + '/leaderboard-streaks?mode=best&offset=0&limit=236')
    leaderboard = response.json()
    for l in leaderboard['leaders']:
        users.add(l['user']['id'])

In [16]:
def parse_post(user_id, post_id, post_title, post_highlights):
    response = session.get('https://api.briefly.co/v1/post/' + post_id + '/sentences')
    sents_from_api = [s for s in response.json() if s['type'] == 'text']
    if len(post_highlights) < 3 or len(post_highlights) < len(sents_from_api) / 7:
        return None
    sents = []
    for s in sents_from_api:
        response = session.get('https://api.briefly.co/v1/sentence/' + s['id'])
        sent_from_api = response.json()
        sent = {
            'id': sent_from_api['id'],
            'text': sent_from_api['sentence'],
            'highlighted': 'users' in sent_from_api and any(u for u in sent_from_api['users'] if u['id'] == user_id),
            'sequence': sent_from_api['sequence'],
            'paragraph': sent_from_api['paragraph']
        }        
        sents.append(sent)
        
    post = {
        'id': post_id,
        'title': post_title,
        'highlighterUser': user_id,
        'sentences': sents
    }
    return post

In [61]:
db = []
for user in ['5f798a28-2037-4025-b8ef-21720b8d4cf8']:
    offset = 0
    while True:
        time.sleep(random.randint(0, 2))
        response = session.get('https://api.briefly.co/v1/user/' + user + '/latest-activity?offset=' + str(offset) + '&limit=100')
        try:
            response.raise_for_status()
        except requests.exceptions.HTTPError:
            break
        posts = response.json()
        if len(posts) == 0:
            break
        for p in posts:
            post = parse_post(user, p['post']['id'], p['post']['title'], p['highlights'])
            if post != None:
                db.append(post)
        offset += len(posts)
with open('briefly.json', 'w') as f:
    json.dump(db, f)        

KeyboardInterrupt: 